In [1]:
import pyspark.sql.functions as F

from pyspark.sql import SparkSession
from pyspark.sql.window import Window

spark = SparkSession.builder.getOrCreate()

data = [
    [1, "IT", 15],
    [2, "Support", 81],
    [3, "Support", 90],
    [4, "Support", 25],
    [5, "IT", 40],
    [6, "IT", 24],
    [7, "Support", 31],
    [8, "Support", 1],
    [9, "HR", 27],
    [10, "IT", 75]
]

windowSpec = Window.partitionBy('department').orderBy('time').rowsBetween(Window.unboundedPreceding, 0)

df = spark.createDataFrame(data, ['time', 'department', 'items_sold'])
df = df.withColumn('cum_sum', F.sum('items_sold').over(windowSpec))

windowSpec = Window.partitionBy('department').orderBy('time')

df = df.withColumn('diff', F.col('cum_sum') - F.lag('cum_sum', default=0).over(windowSpec))
df.show()

+----+----------+----------+-------+----+
|time|department|items_sold|cum_sum|diff|
+----+----------+----------+-------+----+
|   9|        HR|        27|     27|  27|
|   1|        IT|        15|     15|  15|
|   5|        IT|        40|     55|  40|
|   6|        IT|        24|     79|  24|
|  10|        IT|        75|    154|  75|
|   2|   Support|        81|     81|  81|
|   3|   Support|        90|    171|  90|
|   4|   Support|        25|    196|  25|
|   7|   Support|        31|    227|  31|
|   8|   Support|         1|    228|   1|
+----+----------+----------+-------+----+
